# Loading Images with GEE

## Notes

### Images vs ImageCollections

Add this

### Smoke and Cloud Cover in Imagery

## Code

In [1]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='karlzam')

In [11]:
startDate = '2023-09-22'
endDate = '2023-10-24'

In [12]:
Map = geemap.Map()

# Large scale international boundary polygons, simplified
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Using swaziland as an example because it's small
roi = countries.filter(ee.Filter.eq("country_na", "Canada"));

# Get map
Map.addLayer(roi, {}, "Canada")

# Center the object at the region of interest with a zoom of 8
Map.centerObject(roi,4)

# Load in landsat data with filter
# Originally this covers the entire world over a long time series 
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate(startDate, endDate).filterBounds(roi)

# Remove clouds
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat,
    'asFloat': True
})

# Define landsat bands we'll be using 
rgbVis = {'bands': ['B4', 'B3', 'B2'], 'min':0, 'max':0.3}

Map.addLayer(composite.clip(roi), rgbVis, "Landsat RGB")

In [13]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(startDate, endDate)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

Map.add_layer(dataset.mean(), visualization, 'RGB-Sentinel')

In [14]:
# FIRMS
# MODIS fire thermal anomalies 
# Active fire locations 

burned_areas_FIRMS = ee.ImageCollection("FIRMS").filterDate(startDate, endDate)
fires = burned_areas_FIRMS.select('T21')

firesVis = {'min': 0, 'max': 500.0, 'palette': ['red', 'orange', 'yellow']}

Map.addLayer(fires, firesVis, "FIRMS")

In [15]:
# Adds MODIS Burned Area
# Monthly per pixel burned area and quality info (500m)

burned_areas_MODIS = ee.ImageCollection("MODIS/061/MCD64A1").filterDate(startDate, endDate)
burnedAreaMODIS = burned_areas_MODIS.select('BurnDate')
burnedAreaVis = {
  'min': 30.0,
  'max': 341.0,
  'palette': ['4e0400', '951003', 'c61503', 'ff1901'],
}
Map.addLayer(burnedAreaMODIS, burnedAreaVis, 'MODIS')

In [16]:
# Launch map
Map

Map(center=[60.558438200875784, -96.70702570923024], controls=(WidgetControl(options=['position', 'transparent…